## Imports
---

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
from src.heikin_ashi import heikin_ashi
from src.indicators import supertrend, VWAP2, smc, wavetrend3d, atr
import pandas_ta as ta

## Getting the Data
---

In [2]:
auxusd = pd.read_csv("./data_directory/AUXUSD.csv")
auxusd['timestamp'] = pd.to_datetime(auxusd['timestamp'], unit='ms')
auxusd['timestamp'] = pd.to_datetime(auxusd['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
auxusd.set_index('timestamp', inplace=True)
auxusd.drop_duplicates(inplace=True)

In [3]:
eurgbp = pd.read_csv("./data_directory/EURGBP.csv")
eurgbp['timestamp'] = pd.to_datetime(eurgbp['timestamp'], unit='ms')
eurgbp['timestamp'] = pd.to_datetime(eurgbp['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
eurgbp.set_index('timestamp', inplace=True)
eurgbp.drop_duplicates(inplace=True)

In [4]:
forex_data = pd.read_csv("./data_directory/EURUSD_2021_2023.csv")
forex_data['timestamp'] = pd.to_datetime(forex_data['timestamp'], unit='ms')
forex_data['timestamp'] = pd.to_datetime(forex_data['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
forex_data.set_index('timestamp', inplace=True)
forex_data.drop_duplicates(inplace=True)

In [5]:
eurusd = pd.read_csv("./data_directory/EURUSD_NEW.csv")
eurusd.set_index("Date", inplace=True)
eurusd.index = pd.to_datetime(eurusd.index)
eurusd.drop_duplicates(inplace=True)

In [8]:
def Supertrend(df, atr_period, multiplier):
    
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index)



In [14]:
new = Supertrend(forex_data, 10, 2.5)["Final Upperband"]

In [10]:
old = supertrend(forex_data, 10, 2.5, 0)

In [23]:
old.iloc[100:110]

timestamp
2021-06-21 00:54:00         NaN
2021-06-21 00:57:00         NaN
2021-06-21 01:00:00    1.187621
2021-06-21 01:03:00    1.187621
2021-06-21 01:06:00    1.187621
2021-06-21 01:09:00    1.187532
2021-06-21 01:12:00    1.187532
2021-06-21 01:15:00    1.187532
2021-06-21 01:18:00    1.187532
2021-06-21 01:21:00    1.187532
Name: dt, dtype: float64

In [24]:
new.iloc[100:110]

timestamp
2021-06-21 00:54:00         NaN
2021-06-21 00:57:00         NaN
2021-06-21 01:00:00    1.188065
2021-06-21 01:03:00    1.187877
2021-06-21 01:06:00    1.187743
2021-06-21 01:09:00    1.187543
2021-06-21 01:12:00    1.187543
2021-06-21 01:15:00    1.187543
2021-06-21 01:18:00    1.187543
2021-06-21 01:21:00    1.187543
Name: Final Upperband, dtype: float64

In [7]:
# Der choCH wird hier programmiert

def choCH(data, length, band):
    chocolad = pd.Series(index=data.index)
    ob = smc(data, length, band, "30T")
    prevvalue = 0
    orderblocks = []
    i = 0
    if(band == 0):
        for index, values in ob.iteritems():
            if(values > 0 and values != prevvalue):
                prevvalue = values
                give = [i, values]
                orderblocks.append(give)
            i += 1
        for i, (candle, price) in enumerate(orderblocks):
            if price > orderblocks[i-2][1] and orderblocks[i-1][1] < orderblocks[i-2][1] and orderblocks[i-2][1] < orderblocks[i-3][1] and orderblocks[i-3][1] < orderblocks[i-4][1]:
                chocolad.iloc[candle] = price
    elif(band == 1):
        for index, values in ob.iteritems():
            if(values > 0 and values != prevvalue):
                prevvalue = values
                give = [i, values]
                orderblocks.append(give)
            i += 1
        for i, (candle, price) in enumerate(orderblocks):
            if price < orderblocks[i-2][1] and orderblocks[i-1][1] > orderblocks[i-2][1] and orderblocks[i-2][1] > orderblocks[i-3][1] and orderblocks[i-3][1] > orderblocks[i-4][1]:
                chocolad.iloc[candle] = price
    return chocolad
    

hello = choCH(forex_data, 10, 0)




/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_24884/3134265937.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  chocolad = pd.Series(index=data.index)
/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_24884/3134265937.py:10: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, values in ob.iteritems():


In [8]:

for index, value in hello.iteritems():
    if value > 0:
        print(index)

2021-06-20 20:00:00
2021-07-05 04:00:00
2021-07-22 01:00:00
2021-10-14 05:30:00
2022-01-28 15:30:00
2022-02-15 17:30:00
2022-03-07 22:00:00
2022-04-11 06:30:00
2022-04-19 15:00:00
2022-04-28 16:00:00
2022-05-09 06:30:00
2022-05-16 03:00:00
2022-06-30 19:00:00
2022-09-22 15:00:00
2022-10-24 09:30:00
2022-11-29 08:00:00
2022-12-07 16:30:00
2023-01-09 22:00:00
2023-01-19 02:00:00
2023-02-20 00:30:00
2023-02-24 01:30:00


/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_24884/2810368415.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, value in hello.iteritems():


# Kleines Tutorial für Backtesting.py

Es besteht aus einen init part wo alle variablen definiert und initialisiert werden und einen next part wo der algorithmus nach und nach jede candle durchgeht. Dabei kann man die zurzeitige Candle mit __self.data.Close[-1]__ abrufen. Dir davor mit -2 usw. Ihr könnt außerdem dabei zwischen Close, Open, High, Low entscheiden. 

Um die zurzeitige Balance zu bekommen ruft ihr self.equity auf. 

Um Indicatoren hinzuzufügen habt ihr folgende Syntax:

__self.IndicatorName = self.I(Indicator, Parameter1, Parameter2, ...)__

Dabei könnt ihr die jetzigen Werte abrufen wie mit den Daten [-n],  $n \in \mathbb{N}$ .

Um Trades einzugehen bentzt man entweder:

__self.buy(size=deineGroesse)__ 

__self.sell(size=deineGroesse)__

Ihr müsst dabei geachten das man entweder die Share anzahl als Integer übergebt oder die eine Kommazahl von $[0,1)$ die darstellt wie viel Prozent deines Portfolios du reinsetzt hast.

um eine Position zu schließen benutzt ihr: 

__self.position.close()__

Das sind so die wichtigsten Befehle. Leider lässt die libary nicht mehrere Positionen gleichzeitig. Falls ihr das umbedingt braucht kann ich nach einer anderen Libary ausschau halten. 

Falls ihr irgendwelche anderen Befehle braucht wie z.B. eine LimitOrder oder sowas dann findet ihr die unter den Documentations: 

- https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#gsc.tab=0


## The Strategy
---

In [13]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

#Hier die Werte die ihr abändern könnt
shorts = True
longs = True
candlecountEntry = 7 # Nach wie vielen noch geschaut wird ob buy/sell signal kommt
stoplossCandles = 10 # Beim Entry -> Wie viele candles er nach hinten schauen soll für sl
tradehourStart = 6 #UTC Time
tradehourEnd = 15 #UTC Time
wavetrendOn = True
timeframeOBs = "30T"  #T for Minute and H for Hour
timeframeWavetrend = "4H"

class PIES(Strategy):
    def init(self):
        # Store orderblock values
        self.orderblocksU = []
        self.orderblocksB = []
        # If orderblock got hit and price is above/below upperbands/lowerbands
        self.tagged = False
        self.taggedprice = 0
        self.taggeddev = 0
        # Counter of how many candles it was ago when orderblock got tagged
        self.taggedcounter = 0
        # Overall candle count
        self.candlecount = 17
        self.trade_opened = False
        self.stoploss = 0
        self.sholong = 0  #0 stands for long trade and 1 for short trade
        # Values that act as a buffer for the orderblocks after they got hit
        self.orderblocklow = 0
        self.orderblockhigh = 0 
        self.idk = self.data.df
        # Supertrend Indicator
        self.supertrendu = self.I(supertrend, self.data.df, lookback=10, multiplier=2.5, band=0)
        self.supertrendl = self.I(supertrend, self.data.df, lookback=10, multiplier=2.5, band=1)
        # VWAP Indicator
        self.upperOB = self.I(smc, self.data.df, length=10, band=1, timeframe=timeframeOBs)
        self.lowerOB = self.I(smc, self.data.df, length=10, band=0, timeframe=timeframeOBs)
        self.wavetrendMirror = self.I(wavetrend3d, self.idk, 6, timeframeWavetrend, True)
        self.wavetrend = self.I(wavetrend3d, self.idk, 6, timeframeWavetrend, False)
        self.vwap = self.I(VWAP2, self.data.df, 0)
        self.vwapu1 = self.I(VWAP2, self.data.df, 1)
        self.vwapl1 = self.I(VWAP2, self.data.df, 2)
        self.vwapu2 = self.I(VWAP2, self.data.df, 3)
        self.vwapl2 = self.I(VWAP2, self.data.df, 4)
        self.vwapu3 = self.I(VWAP2, self.data.df, 5)
        self.vwapl3 = self.I(VWAP2, self.data.df, 6)
        self.prevvalueU = 0
        self.prevvalueB = 0
        # ATR Indicator
        self.atr = self.I(atr, self.data.df)

    def next(self):
        # Add orderblocks
        if(self.upperOB[-1] > 0 and self.upperOB[-1] != self.prevvalueU):
            self.orderblocksU.append([self.data.High[-1], self.data.Low[-1], self.candlecount])
            self.prevvalueU = self.upperOB[-1]
        if(self.lowerOB[-1] > 0 and self.lowerOB[-1] != self.prevvalueB):
            self.orderblocksB.append([self.data.High[-1], self.data.Low[-1], self.candlecount])
            self.prevvalueB = self.lowerOB[-1]
        # If no trade is open
        if not self.trade_opened and self.data.index[-1].hour >= tradehourStart and self.data.index[-1].hour <= tradehourEnd:
            # If it didnt hit orderblock + upper/lower band yet
            if self.tagged==False:
                # For loops to iterate through the orderblocks and check if they got hit
                if shorts:
                    if self.wavetrendMirror[-1] > self.wavetrend[-1] or wavetrendOn == False:
                        for high, low, candle in self.orderblocksU:
                            if self.data.High[-1] > low and self.data.High[-1] < high and self.candlecount - candle < 1000 and ((self.data.High[-1] > self.vwapu2[-1] - (self.atr[-1]/2) and self.data.High[-1] < self.vwapu2[-1] + (self.atr[-1]/2)) or (self.data.High[-1] > self.vwapu3[-1] - (self.atr[-1]/2) and self.data.High[-1] < self.vwapu3[-1] + (self.atr[-1]/2))):
                                self.tagged = True
                                if(self.data.High[-1] < self.vwapu2[-1] + (self.atr[-1]/2)):
                                    self.taggeddev = 2
                                else:
                                    self.taggeddev = 3
                                self.taggedprice = self.data.High[-1]
                                self.orderblocklow = low
                                self.sholong = 1
                                self.orderblocksU.remove([high, low, candle])
                                break
                if longs:
                    if self.wavetrendMirror[-1] < self.wavetrend[-1] or wavetrendOn == False:
                        for high, low, candle in self.orderblocksB:
                            if self.data.Low[-1] < high and self.data.Low[-1] > low and self.candlecount - candle < 1000 and ((self.data.Low[-1] < self.vwapl2[-1] + (self.atr[-1]/2) and self.data.Low[-1] > self.vwapl2[-1] - (self.atr[-1]/2)) or (self.data.Low[-1] < self.vwapl3[-1] + (self.atr[-1]/2) and self.data.Low[-1] > self.vwapl3[-1] - (self.atr[-1]/2))):
                                self.tagged = True
                                if(self.data.Low[-1] > self.vwapl2[-1] - (self.atr[-1]/2)):
                                    self.taggeddev = 2
                                else:
                                    self.taggeddev = 3
                                self.orderblockhigh = high
                                self.sholong = 0
                                self.orderblocksB.remove([high, low, candle])
                                break

            # If it hit and we have to go short
            elif (self.tagged == True and self.sholong == 1):
                # Heikin Ashi Candles)
                ha_close = 1/4*(self.data.Open[-1] + self.data.Close[-1] + self.data.High[-1] + self.data.Low[-1])
                ha_open = 1/2*(self.data.Open[-2] + self.data.Close[-2])
                ha_high = max(self.data.High[-1], ha_open, ha_close)
                ha_low = min(self.data.Low[-1], ha_open, ha_close)
                if((self.data.High[-1] > self.vwapu2[-1] + (self.atr[-1]/2) and self.taggeddev == 2) or (self.data.High[-1] > self.vwapu3[-1] + (self.atr[-1]/2) and self.taggeddev == 3)):
                    self.tagged = False
                else:
                    if self.taggedcounter < candlecountEntry:
                        if self.data.High[-1] < self.orderblocklow and ha_open > (ha_high - ((ha_high - ha_low)*0.12) and self.data.Close[-1]>self.vwapu1[-1]):
                            self.trade_opened = True
                            self.stoploss = 0
                            for i in range(1, stoplossCandles):
                                if self.data.High[-i] > self.stoploss:
                                    self.stoploss = self.data.High[-i]
                            def berechnen(kapital, stoploss, entry):
                                    diff = stoploss - entry
                                    size = abs((kapital/10000) / diff)
                                    return int(round(size,0))
                            sizeO = berechnen(self.equity, self.stoploss, self.data.Close[-1])
                            try:
                                self.sell(size=sizeO)
                            except:
                                print("Something went wrong")
                        self.taggedcounter += 1
                    else: 
                        self.tagged = False
                        self.taggedcounter = 0
            # If it hit and we have to go long
            elif (self.tagged == True and self.sholong == 0):
                # Heikin Ashi Candles
                ha_close = 1/4*(self.data.Open[-1] + self.data.Close[-1] + self.data.High[-1] + self.data.Low[-1])
                ha_open = 1/2*(self.data.Open[-2] + self.data.Close[-2])
                ha_high = max(self.data.High[-1], ha_open, ha_close)
                ha_low = min(self.data.Low[-1], ha_open, ha_close)
                if((self.data.Low[-1] < self.vwapl2[-1] - (self.atr[-1]/2) and self.taggeddev == 2) or (self.data.Low[-1] < self.vwapl3[-1] - (self.atr[-1]/2) and self.taggeddev == 3)):
                    self.tagged = False
                if self.taggedcounter < candlecountEntry:
                    if self.data.Low[-1] > self.orderblockhigh and ha_open < (ha_low + (ha_high - ha_low) * 0.12) and self.data.Close[-1]<self.vwapl1[-1]:
                        self.stoploss = 100 # Set stoploss to high value
                        for i in range(1, stoplossCandles):
                            if self.data.Low[-i] < self.stoploss:
                                self.stoploss = self.data.Low[-i]
                        def berechnen(kapital, stoploss, entry):
                            diff = stoploss - entry
                            size = abs((kapital/10000) / diff)
                            return int(round(size,0))
                        sizeO = berechnen(self.equity, self.stoploss, self.data.Close[-1])
                        try:    
                            self.buy(size=sizeO)
                        except:
                            print("Something went wrong")
                        self.trade_opened = True
                    self.taggedcounter += 1
                else: 
                    self.tagged = False
                    self.taggedcounter = 0
        
        elif self.trade_opened: # Stop loss path
            if self.sholong == 0:
                if(self.supertrendl[-2] > 0 and self.supertrendu[-1] > 0) or self.data.Close[-1] < self.stoploss:
                    self.position.close()
                    self.trade_opened = False
                    self.tagged = False
            elif self.sholong == 1:
                if(self.supertrendu[-2] > 0 and self.supertrendl[-1] > 0) or self.data.Close[-1] > self.stoploss:
                    self.position.close()
                    self.trade_opened = False
                    self.tagged = False
        self.candlecount = self.candlecount + 1
        
bt2 = Backtest(eurusd, PIES, cash=1000000, commission=0.00, exclusive_orders=True)
# Run the backtest and append the results to the list
output2 = bt2.run()
# Print the results
print(output2)

bt2.plot(filename="./ChartPlots/new_dataset/PIES_5.html", plot_volume=False)

KeyboardInterrupt: 

In [46]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Parameter für Philip

atr_window = 6
einsatz = 0.5 # Hier stehen Kommazahlen für den Prozentuallen Anteil deiner Equity die du setzen willst
# Oder du kannst auch ganze Zahlen nuzen für die Anzahl der Shares die du kaufen willst. Falls du alles reinsetzen
# Willst benutz 0.9999
satr = 3 # * atr in stoploss
tatr = 9 # * atr in take profit



class PIIL(Strategy):
    def init(self):
        self.trade_opened = False
        self.take_profit = 0
        self.sl = 0
        
        self.atr = self.I(atr, self.data.df, atr_window)

    def next(self):
        if not self.trade_opened and self.data.index[-1].hour == 5 and self.data.index[-1].minute == 0:
            self.buy(size=einsatz)
            self.sl = self.data.Close[-1] - satr * self.atr[-1]
            self.take_profit = self.data.Close[-1] + tatr * self.atr[-1]
            self.trade_opened = True

        elif self.trade_opened:
            if self.data.Close[-1] > self.take_profit or self.data.Close[-1] < self.sl:
                self.position.close()
                self.trade_opened = False
                self.take_profit = 0
                self.sl = 0


class PIIS(Strategy):
    def init(self):
        self.candlecount = 0
        self.trade_opened = False
        self.take_profit = 0
        self.sl = 0
        self.atr = self.I(atr, self.data.df)

    def next(self):
        if not self.trade_opened and self.data.index[-1].hour == 5 and self.data.index[-1].minute == 0:
            self.sl = self.data.Close[-1] + satr*self.atr[-1]
            self.take_profit = self.data.Close[-1] - tatr*self.atr[-1]
            self.sell(size=einsatz)
            self.trade_opened = True

        elif self.trade_opened:
            if self.data.Close[-1] < self.take_profit or self.data.Close[-1] > self.sl:
                self.position.close()
                self.trade_opened = False
                self.take_profit = 0
                self.sl = 0

bt = Backtest(eurusd, PIIL, cash=1000000, commission=0.00, exclusive_orders=True)

output = bt.run()
print(output)
bt.plot()


Start                     2000-01-03 00:00:00
End                       2019-12-31 22:57:00
Duration                   7302 days 22:57:00
Exposure Time [%]                    14.40731
Equity Final [$]                 964314.55851
Equity Peak [$]                  1018243.2493
Return [%]                          -3.568544
Buy & Hold Return [%]               11.291795
Return (Ann.) [%]                   -0.176011
Volatility (Ann.) [%]                1.520985
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -7.924997
Avg. Drawdown [%]                   -0.275629
Max. Drawdown Duration     7243 days 14:18:00
Avg. Drawdown Duration      151 days 23:58:00
# Trades                                 3876
Win Rate [%]                        27.012384
Best Trade [%]                       1.379617
Worst Trade [%]                     -1.811289
Avg. Trade [%]                    

/Users/dennisschielke/miniconda3/envs/pie/lib/python3.8/site-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '1D'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/Users/dennisschielke/miniconda3/envs/pie/lib/python3.8/site-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
/Users/dennisschielke/miniconda3/envs/pie/lib/python3.8/site-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
/Users/dennisschielke/miniconda3/envs/pie/lib/python3.8/site-pa

GridPlot(id='p2679', ...)

In [14]:
output2 = bt2.run()
print(output2)

Start                     2021-06-20 19:00:00
End                       2023-05-19 21:03:00
Duration                    698 days 02:03:00
Exposure Time [%]                   15.730855
Equity Final [$]                 996656.89703
Equity Peak [$]                 1013179.36952
Return [%]                           -0.33431
Buy & Hold Return [%]               -8.775154
Return (Ann.) [%]                   -0.130344
Volatility (Ann.) [%]                 1.45411
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                     -2.6457
Avg. Drawdown [%]                   -0.105197
Max. Drawdown Duration      333 days 15:27:00
Avg. Drawdown Duration        8 days 01:41:00
# Trades                                  486
Win Rate [%]                         26.54321
Best Trade [%]                       0.845836
Worst Trade [%]                      -0.53448
Avg. Trade [%]                    